### CFTC CoT Dashboard (Data cleaning and Transformation) 

In [29]:
### find repo root automatically, then load parquet

import pandas as pd
from pathlib import Path
import os

## Find project root robustly

def find_project_root(project_name="cftc-cot-dashboard") -> Path:
    """
    Walk upward from current working directory until we find the repo folder.
    """
    p = Path.cwd().resolve()
    for candidate in [p] + list(p.parents):
        if candidate.name == project_name:
            return candidate
        if (candidate / project_name).is_dir():
            return candidate / project_name
    raise FileNotFoundError(
        f"Could not find '{project_name}' above current directory: {Path.cwd()}"
    )

PROJECT_ROOT = find_project_root()
print("CWD:", Path.cwd())
print("PROJECT_ROOT:", PROJECT_ROOT)

## Define input path

IN_PATH = PROJECT_ROOT / "data" / "processed" / "cot_combined_tff_disagg.parquet"
print("IN_PATH:", IN_PATH)
print("Exists:", IN_PATH.exists())

if not IN_PATH.exists():
    raise FileNotFoundError(
        f"Expected parquet not found at: {IN_PATH}\n"
        f"Tip: list available parquet files with:\n"
        f"  list((PROJECT_ROOT/'data').rglob('*.parquet'))"
    )

## Load + inspect schema

df = pd.read_parquet(IN_PATH)

print("\nShape:", df.shape)
print("\nColumns (count={}):".format(len(df.columns)))
for c in df.columns:
    print(" -", c)

print("\nPreview:")
display(df.head(3))


CWD: /Users/qiaoenn/Desktop/cftc-cot-dashboard/notebooks
PROJECT_ROOT: /Users/qiaoenn/Desktop/cftc-cot-dashboard
IN_PATH: /Users/qiaoenn/Desktop/cftc-cot-dashboard/data/processed/cot_combined_tff_disagg.parquet
Exists: True

Shape: (90678, 183)

Columns (count=183):
 - id
 - market_and_exchange_names
 - report_date_as_yyyy_mm_dd
 - yyyy_report_week_ww
 - contract_market_name
 - cftc_contract_market_code
 - cftc_market_code
 - cftc_region_code
 - cftc_commodity_code
 - commodity_name
 - open_interest_all
 - dealer_positions_long_all
 - dealer_positions_short_all
 - dealer_positions_spread_all
 - asset_mgr_positions_long
 - asset_mgr_positions_short
 - asset_mgr_positions_spread
 - lev_money_positions_long
 - lev_money_positions_short
 - lev_money_positions_spread
 - other_rept_positions_long
 - other_rept_positions_short
 - other_rept_positions_spread
 - tot_rept_positions_long_all
 - tot_rept_positions_short
 - nonrept_positions_long_all
 - nonrept_positions_short_all
 - change_in_open

,id,market_and_exchange_names,report_date_as_yyyy_mm_dd,yyyy_report_week_ww,contract_market_name,cftc_contract_market_code,cftc_market_code,cftc_region_code,cftc_commodity_code,commodity_name,...,conc_net_le_8_tdr_long_old,conc_net_le_8_tdr_short_old,conc_gross_le_4_tdr_long_2,conc_gross_le_4_tdr_short_2,conc_gross_le_8_tdr_long_2,conc_gross_le_8_tdr_short_2,conc_net_le_4_tdr_long_other,conc_net_le_4_tdr_short_other,conc_net_le_8_tdr_long_other,conc_net_le_8_tdr_short_other
0,220913020601F,UST BOND - CHICAGO BOARD OF TRADE,2022-09-13T00:00:00.000,2022 Report Week 37,UST BOND,020601,CBT,CHI,020,T-BONDS,...,None,None,None,None,None,None,None,None,None,None
1,220913020604F,ULTRA UST BOND - CHICAGO BOARD OF TRADE,2022-09-13T00:00:00.000,2022 Report Week 37,ULTRA UST BOND,020604,CBT,CHI,020,T-BONDS,...,None,None,None,None,None,None,None,None,None,None
2,220913042601F,UST 2Y NOTE - CHICAGO BOARD OF TRADE,2022-09-13T00:00:00.000,2022 Report Week 37,UST 2Y NOTE,042601,CBT,CHI,042,"T-NOTES, 1-2 YEAR",...,None,None,None,None,None,None,None,None,None,None


In [30]:
### standardise core identifier columns 

base = df.rename(columns={
    "report_date_as_yyyy_mm_dd": "date",
    "contract_market_name": "market",
    "cftc_contract_market_code": "cftc_code",
    "open_interest_all": "open_interest",
}).copy()

### Clean / cast types

base["date"] = pd.to_datetime(base["date"], errors="coerce")

base["market"] = (
    base["market"]
    .astype(str)
    .str.strip()
)

base["cftc_code"] = (
    base["cftc_code"]
    .astype(str)
    .str.strip()
)

base["open_interest"] = pd.to_numeric(
    base["open_interest"],
    errors="coerce"
)

## sanity checks

assert base["date"].notna().all(), "Some dates failed to parse"
assert base["cftc_code"].ne("").all(), "Empty cftc_code detected"

print(base[["date", "market", "cftc_code", "open_interest"]].head(3))

        date          market cftc_code  open_interest
0 2022-09-13        UST BOND    020601        1225024
1 2022-09-13  ULTRA UST BOND    020604        1376937
2 2022-09-13     UST 2Y NOTE    042601        2008780


In [31]:
### Build tidy dataframe (Leveraged Funds only)

### filter to rows where Leveraged Funds data actually exists (TFF)

tff = base[base["source"].astype(str).str.contains("tff", case=False, na=False)].copy()
print("Rows total:", len(base), "| Rows after TFF filter:", len(tff))

### Build Leveraged Funds tidy frame from TFF only

lf = tff[[
    "date",
    "market",
    "cftc_code",
    "open_interest",
    "lev_money_positions_long",
    "lev_money_positions_short",
    "lev_money_positions_spread",
]].copy()

# Standardise names to the tidy schema

lf = lf.rename(columns={
    "lev_money_positions_long": "long",
    "lev_money_positions_short": "short",
    "lev_money_positions_spread": "spreading",
})

### Add group label

lf["group"] = "Leveraged Funds"

### Robust numeric cleaning (handles strings like "12,345")
def to_num(s):
    return pd.to_numeric(
        s.astype(str).str.replace(",", "", regex=False).replace("nan", pd.NA),
        errors="coerce"
    )

### Ensure numeric

for c in ["open_interest", "long", "short", "spreading"]:
    lf[c] = pd.to_numeric(lf[c], errors="coerce")

### Derived metrics

lf["net"] = lf["long"] - lf["short"]

### Avoid divide-by-zero / missing OI

oi = lf["open_interest"]
bad_oi = oi.isna() | (oi <= 0)

lf["pct_oi_net"] = lf["net"] / oi
lf["pct_oi_long"] = lf["long"] / oi
lf["pct_oi_short"] = lf["short"] / oi

lf.loc[bad_oi, ["pct_oi_net", "pct_oi_long", "pct_oi_short"]] = pd.NA

### Quick sanity check: are there NaNs?

print("Non-null counts:",
      lf[["long","short","spreading","net"]].notna().sum().to_dict())

### reorder columns nicely
lf = lf[[
    "date",
    "market",
    "cftc_code",
    "group",
    "long",
    "short",
    "spreading",
    "open_interest",
    "net",
    "pct_oi_net",
    "pct_oi_long",
    "pct_oi_short",
]]

### sort for time-series friendliness
lf = lf.sort_values(["cftc_code", "date"]).reset_index(drop=True)

print("Rows:", len(lf))
display(lf.head(10000))


Rows total: 90678 | Rows after TFF filter: 26912
Non-null counts: {'long': 26912, 'short': 26912, 'spreading': 26912, 'net': 26912}
Rows: 26912


,date,market,cftc_code,group,long,short,spreading,open_interest,net,pct_oi_net,pct_oi_long,pct_oi_short
0,2006-06-13,UST BOND,020601,Leveraged Funds,147085,103720,23018,762787,43365,0.056851,0.192826,0.135975
1,2006-06-20,UST BOND,020601,Leveraged Funds,134618,111075,10343,747700,23543,0.031487,0.180043,0.148556
2,2006-06-27,UST BOND,020601,Leveraged Funds,125196,130793,11073,765619,-5597,-0.007310,0.163523,0.170833
3,2006-07-03,UST BOND,020601,Leveraged Funds,116813,110063,10055,743619,6750,0.009077,0.157087,0.148010
4,2006-07-11,UST BOND,020601,Leveraged Funds,111782,93077,13151,753141,18705,0.024836,0.148421,0.123585
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2009-05-12,BRITISH POUND,096742,Leveraged Funds,11783,15109,31,91329,-3326,-0.036418,0.129017,0.165435
9996,2009-05-19,BRITISH POUND,096742,Leveraged Funds,9908,15509,347,92870,-5601,-0.060310,0.106687,0.166997
9997,2009-05-26,BRITISH POUND,096742,Leveraged Funds,10383,12401,1651,93484,-2018,-0.021587,0.111067,0.132654
9998,2009-06-02,BRITISH POUND,096742,Leveraged Funds,12369,10307,2059,99293,2062,0.020767,0.124571,0.103804


In [32]:
WINDOW = 260  # ~5 years weekly

### rolling percentiles (5Y window) 

def rolling_percentile(s: pd.Series, window: int):
    # percentile rank of the latest value inside each rolling window
    return s.rolling(window, min_periods=26).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1],
        raw=False
    )

lf = lf.sort_values(["cftc_code", "date"]).copy()

lf["long_pctile_5y"] = lf.groupby("cftc_code")["long"].transform(lambda s: rolling_percentile(s, WINDOW))
lf["short_pctile_5y"] = lf.groupby("cftc_code")["short"].transform(lambda s: rolling_percentile(s, WINDOW))
lf["net_pctile_5y"] = lf.groupby("cftc_code")["net"].transform(lambda s: rolling_percentile(s, WINDOW))


### net trend features 
## net_ma_13 = 13-week moving average (smooth trend)
## net_chg_13 = change over 13 weeks (momentum-ish)

lf["net_ma_13"] = lf.groupby("cftc_code")["net"].transform(lambda s: s.rolling(13, min_periods=1).mean())
lf["net_chg_13"] = lf.groupby("cftc_code")["net"].transform(lambda s: s.diff(13))


In [33]:
OUT_PATH = PROJECT_ROOT / "data" / "processed" / "cot_tidy_leveraged_funds.parquet"
lf.to_parquet(OUT_PATH, index=False)
print("Saved:", OUT_PATH)


Saved: /Users/qiaoenn/Desktop/cftc-cot-dashboard/data/processed/cot_tidy_leveraged_funds.parquet


In [34]:
### create latest snapshot table

latest_date = lf["date"].max()
latest = lf[lf["date"] == latest_date].copy()

LATEST_PATH = PROJECT_ROOT / "data" / "processed" / "cot_latest_leveraged_funds.parquet"
latest.to_parquet(LATEST_PATH, index=False)

print("Latest date:", latest_date)
print("Saved:", LATEST_PATH, "| rows:", len(latest))


Latest date: 2026-02-03 00:00:00
Saved: /Users/qiaoenn/Desktop/cftc-cot-dashboard/data/processed/cot_latest_leveraged_funds.parquet | rows: 31


own remarks: 

1. long = Number of long contracts, shows absolute bullish exposure
2. short = Number of short contracts, represents bearish exposure or hedging pressure
3. spreading = Contracts where traders are long one maturity and short another (calendar spreads) (betting on the difference between two related contracts)
- High spreading → positioning driven by curve structure
- Low spreading → positioning more directional
4. open interest = Total outstanding contracts in the market, measures market participation and liquidity
- Rising OI + rising price → new money entering trend
- Falling OI → position unwinds / trend exhaustion
5. net = long - short (directional bias)
- Positive net → net bullish
- Negative net → net bearish
6. pct_oi_long = long / OI (shows how dominant long exposure is relative to the whole market)
7. pct_oi_short = short / OI (Measures short pressure relative to total participation)
8. pct_oi_net = net / OI
9. long_pctile_5y = Where current longs sit relative to the past 5 years (percentile rank of this week’s long position compared to the past ~5 years of longs for the same contract)(0 = extremely low, 1 = extremely high)
10. short_pctile_5y = 5-year percentile rank of shorts, flags crowded bearish trades
11. net_pctile_5y = 5-year percentile of net positioning
12. net_ma_13 = 13-week moving average of net positioning (smooths noisy weekly CoT data to reveal structural trend)
- Rising MA → sustained accumulation
- Falling MA → sustained unwind
13. net_chg_13 = Change in net positioning over the last 13 weeks, measures positioning momentum